In [1]:
import numpy as np
import pandas as pd
import os
import torch
from scipy import stats
from statsmodels.stats.multitest import multipletests
import scanpy as sc
import matplotlib.pyplot as plt

In [2]:
to_save_dir="../edges/"
data_dir="../../data/AD/processed/"
genes = torch.load("/".join(data_dir.split("/")[:-2]) + "/genes.pth")


cell_types=['Astrocyte', 'Chandelier', 'Endothelial', 'L2/3 IT', 'L4 IT', 'L5 ET', 'L5 IT', 'L5/6 NP', 'L6 CT', 'L6 IT', 'L6 IT Car3', 'L6b', 'Lamp5', 'Lamp5 Lhx6', 'Microglia-PVM', 'OPC', 'Oligodendrocyte', 'Pax6', 'Pvalb', 'Sncg', 'Sst', 'Sst Chodl', 'VLMC', 'Vip']

samples=['H20.33.001.CX28.MTG.02.007.1.02.02', 'H20.33.001.CX28.MTG.02.007.1.02.03', 'H20.33.001.Cx28.MTG.02.007.1.01.03', 'H20.33.004.Cx26.MTG.02.007.1.01.04', 'H20.33.004.Cx26.MTG.02.007.1.01.05', 'H20.33.004.Cx26.MTG.02.007.1.02.02', 'H20.33.004.Cx26.MTG.02.007.1.02.03', 'H20.33.004.Cx26.MTG.02.007.1.02.04', 'H20.33.012.Cx24.MTG.02.007.1.01.01', 'H20.33.012.Cx24.MTG.02.007.1.03.02', 'H20.33.012.Cx24.MTG.02.007.1.03.03', 'H20.33.015.CX24.MTG.02.007.1.03.01', 'H20.33.015.CX24.MTG.02.007.1.03.02', 'H20.33.015.Cx24.MTG.02.007.1.03.03', 'H20.33.025.Cx28.MTG.02.007.1.01.02', 'H20.33.025.Cx28.MTG.02.007.1.01.04', 'H20.33.025.Cx28.MTG.02.007.1.01.06', 'H20.33.035.Cx26.MTG.02.007.1.01.03', 'H20.33.035.Cx26.MTG.02.007.1.01.04', 'H20.33.040.Cx25.MTG.02.007.1.01.03', 'H20.33.040.Cx25.MTG.02.007.1.01.04', 'H20.33.044.Cx26.MTG.02.007.1.01.02', 'H20.33.044.Cx26.MTG.02.007.1.01.03', 'H20.33.044.Cx26.MTG.02.007.1.01.04', 'H21.33.001.Cx22.MTG.02.007.1.01.03', 'H21.33.001.Cx22.MTG.02.007.1.01.04', 'H21.33.005.Cx18.MTG.02.007.02.03', 'H21.33.005.Cx18.MTG.02.007.02.04', 'H21.33.006.Cx28.MTG.02.007.1.01.09.02', 'H21.33.006.Cx28.MTG.02.007.1.01.09.03', 'H21.33.006.Cx28.MTG.02.007.1.01.09.04', 'H21.33.011.Cx26.MTG.02.007.3.01.04', 'H21.33.011.Cx26.MTG.02.007.3.01.05', 'H21.33.011.Cx26.MTG.02.007.3.01.06', 'H21.33.012.Cx26.MTG.02.007.1.01.04', 'H21.33.012.Cx26.MTG.02.007.1.01.05', 'H21.33.012.Cx26.MTG.02.007.1.01.06', 'H21.33.013.Cx24.MTG.02.007.1.06', 'H21.33.014.CX26.MTG.02.007.1.02.02', 'H21.33.015.Cx26.MTG.02.007.1.0', 'H21.33.015.Cx26.MTG.02.007.1.1', 'H21.33.015.Cx26.MTG.02.007.1.2', 'H21.33.016.Cx26.MTG.02.007.3.01.01', 'H21.33.016.Cx26.MTG.02.007.3.01.02', 'H21.33.016.Cx26.MTG.02.007.3.01.03', 'H21.33.019.Cx30.MTG.02.007.5.0', 'H21.33.019.Cx30.MTG.02.007.5.01.01', 'H21.33.019.Cx30.MTG.02.007.5.01.02', 'H21.33.021.Cx26.MTG.02.007.1.04', 'H21.33.021.Cx26.MTG.02.007.1.06', 'H21.33.022.Cx26.MTG.02.007.2.M.02', 'H21.33.022.Cx26.MTG.02.007.2.M.03', 'H21.33.022.Cx26.MTG.02.007.2.M.04', 'H21.33.023.Cx26.MTG.02.007.1.03.01', 'H21.33.023.Cx26.MTG.02.007.1.03.04', 'H21.33.023.Cx26.MTG.02.007.1.03.05', 'H21.33.025.CX26.MTG.02.007.4.01.02', 'H21.33.025.CX26.MTG.02.007.4.01.04', 'H21.33.025.CX26.MTG.02.007.4.01.06', 'H21.33.028.CX28.MTG.02.007.1.01.01', 'H21.33.028.Cx28.MTG.02.007.1.02.02', 'H21.33.028.Cx28.MTG.02.007.1.02.04', 'H21.33.031.CX24.MTG.02.007.1.01.01', 'H21.33.031.CX24.MTG.02.007.1.01.02', 'H21.33.032.CX24.MTG.02.007.1.01.04', 'H21.33.038.Cx20.MTG.02.007.3.01.02', 'H21.33.038.Cx20.MTG.02.007.3.01.04', 'H21.33.040.Cx22.MTG.02.007.3.03.01', 'H21.33.040.Cx22.MTG.02.007.3.03.03', 'H21.33.040.Cx22.MTG.02.007.3.03.04']

cell_type_pair_sequence=[]
for cell_typei in cell_types:
    for cell_typej in cell_types:
        cell_type_pair_sequence.append(cell_typei+"__"+cell_typej)

meta_rank={
  "Overall AD neuropathological Change": {"Not AD": 1, "Low": 2, "Intermediate": 3, "High": 4},
  "Thal": {"Thal 0": 1, "Thal 1": 2, "Thal 2": 3, "Thal 3": 4, "Thal 4": 5, "Thal 5": 6},
  "Braak": {"Braak 0": 1, "Braak II": 2, "Braak III": 3, "Braak IV": 4, "Braak V": 5, "Braak VI": 6},
  "CERAD score": {"Absent": 1, "Sparse": 2, "Moderate": 3, "Frequent": 4},
  "Overall CAA Score": {"Not identified": 1, "Mild": 2, "Moderate": 3},
  "Highest Lewy Body Disease": {
    "Not Identified (olfactory bulb not assessed)": 1,
    "Not Identified (olfactory bulb assessed)": 2,
    "Olfactory bulb only": 3,
    "Brainstem-predominant": 4,
    "Limbic (Transitional)": 5,
    "Neocortical (Diffuse)": 6
  },
  "Atherosclerosis": {"None": 1, "Mild": 2, "Moderate": 3},
  "Arteriolosclerosis": {"Mild": 1, "Moderate": 2, "Severe": 3},
  "LATE": {"Not Identified": 1, "LATE Stage 1": 2, "LATE Stage 2": 3, "LATE Stage 3": 4},
  "Cognitive Status": {"No dementia": 1, "Dementia": 2}
}

cell_type_pair_sequence=[]
for cell_typei in cell_types:
    for cell_typej in cell_types:
        cell_type_pair_sequence.append(cell_typei+"__"+cell_typej)

In [3]:
metadata_df = pd.read_csv('metadata.csv', index_col=0)
common_samples=list(set(samples).intersection(set(metadata_df.index.tolist())))

useful_sample=[i for i in samples if i in common_samples]
useful_index=[(i in useful_sample) for i in samples]

In [4]:
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests

def filter_z_score(data1, type_I_cutoff=0.05, type_II_cutoff=0.05):
    data=np.mean(data1,axis=0,keepdims=True)*np.sqrt(data1.shape[0])
    # Convert z-scores to p-values (two-tailed)
    p_values = 2 * (1 - norm.cdf(np.abs(data)))

    # Shape of data
    b, t, g = data.shape

    # Initialize an array to hold adjusted p-values
    adjusted_p_values = np.zeros_like(data)
    p_cutoffs = np.zeros_like(data)

    # Adjust p-values using the BH method along axis=1
    for i in range(b):
        for j in range(g):
            # Extract the p-values for this particular set
            original_p_values = p_values[i, :, j]
            # Apply BH correction
            _, adjusted_pvals, _, _ = multipletests(original_p_values, alpha=0.05, method='fdr_bh')
            adjusted_p_values[i, :, j] = adjusted_pvals
            
            p_adj_cutoffj=np.max(original_p_values[adjusted_pvals<type_I_cutoff])
            p_cutoffs[i, :, j]=np.ones_like(adjusted_pvals)*p_adj_cutoffj
            
    alphas=np.abs(norm.ppf(p_cutoffs.flatten()/2).reshape(p_cutoffs.shape))
    type_II_error=np.abs(norm.cdf(-np.abs(data)+alphas)-norm.cdf(-np.abs(data)-alphas))
    flags=(adjusted_p_values > type_I_cutoff) | (type_II_error > type_II_cutoff)
    
    # Set z-scores to 0 where the adjusted p-values are < 0.05
    data[flags] = 0
    return data

In [9]:
probablity_z=np.load("./z_regressionp/z_values.npy")[useful_index,:,:]
strength=np.abs(np.load("./z_strength_sign1/z_values.npy")[useful_index,:,:])
print(probablity_z.shape,strength.shape)

probablity_z=filter_z_score(probablity_z)
strength=np.where(probablity_z!=0,np.abs(strength),np.zeros_like(strength))
strength=np.where(probablity_z>0,strength,-strength)
meta=metadata_df.loc[useful_sample,'Cognitive Status'].values

(69, 576, 140) (69, 576, 140)


In [10]:
probablity_z=probablity_z.squeeze(axis=0)
df=pd.DataFrame(data=probablity_z,columns=genes,index=cell_type_pair_sequence)
df.to_csv("./Aggregated/all.csv")

In [11]:
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

def group_comparison(data, meta, cutoff=0.1):
    # Extract unique groups
    groups = np.unique(meta)
    if len(groups) != 2:
        raise ValueError("Meta should contain exactly two unique groups.")
    
    group1_indices = np.where(meta == groups[0])[0]
    group2_indices = np.where(meta == groups[1])[0]
    
    s, t, c = data.shape
    p_values = np.zeros((t, c))
    
    # Perform Wilcoxon rank sum tests
    for i in range(t):
        for j in range(c):
            group1_data = data[group1_indices, i, j]
            group2_data = data[group2_indices, i, j]
            
            if np.all(group1_data == 0) and np.all(group2_data == 0):
                p_values[i, j] = 1
            else:
                _, p_val = stats.ranksums(group1_data, group2_data)
                p_values[i, j] = p_val
    
    # Adjust p-values using FDR-BH
    adjusted_p_values = np.zeros_like(p_values)
    for j in range(c):
        valid_p = p_values[:, j] < 1  # Only consider p-values < 1 for adjustment
        adjusted_p_values[valid_p, j] = multipletests(p_values[valid_p, j], method='fdr_bh')[1]

    # Calculate mean differences
    mean_diff = data[group1_indices].mean(axis=0) - data[group2_indices].mean(axis=0)
    
    # Zero out mean differences where adjusted p-values are above the cutoff
    mean_diff[adjusted_p_values > cutoff] = 0
    
    return mean_diff

diff=group_comparison(strength,meta)

In [8]:
df_diff=pd.DataFrame(data=diff,columns=genes,index=cell_type_pair_sequence)
df_diff.to_csv("./Aggregated/diff.csv")